## 8.2 LiveData - podstawy

Kontynuujemy aplikację z poprzedniego notatnika - dodamy `LiveData` do aplikacji i rozpoczniemy aktualizację danych na ui.

`LiveData` przechowuje dane, które inne obiekty mogą obserwować i reagować na zmiany. Jest to element tzw. **lifecycle-aware** - gdy podłączamy obserwator do `LiveData`, jest on powiązany z obiektem `LifeCycleOwner` (aktywność/fragment) i wykonuje aktualizacje tylko w stanie aktywnym.

<img src="https://media1.giphy.com/media/tcOAkShGdPpbMpwlMw/giphy.gif" width="150" />

Zmiany rozpocznijmy od dodania `MutableLiveData` do klasy `ScrambleViewModel`, wpierw zmodyfikujmy `currentScrambledWord`.

In [ ]:
private final MutableLiveData<String> currentScrambledWord = 
    new MutableLiveData<>();

Przejdźmy do gettera i zwróćmy obiekt `LiveData` - niemutowalny - wystawiamy tylko dane niemutowalne

In [ ]:
public LiveData<String> getCurrentScrambledWord() {
        return currentScrambledWord;
    }

Następnioe musimy zmodyfikować metodę `getNextWord` w klasie `ScrambleViewModel` - wartość zmiennej `currentScrambledWord` teraz modyfikujemy przez wywołanie metody `setValue`

In [ ]:
    currentScrambledWord.setValue(
        Arrays.toString(tempWord)
            .replace(",", "")
            .replace("[", "")
            .replace("]", "")
            .replace(" ", "")
            .trim());

Kolejne dwie zmienne, które musimy obserwować we fragmencie to `currentWordCount` i `score` - dodajmy `MutableLiveData` i ustawmy odpowiednie gettery

In [ ]:
    private final MutableLiveData<Integer> currentWordCount = 
        new MutableLiveData<>(0); // inicjujemy 0
    private final MutableLiveData<Integer> score = 
        new MutableLiveData<>(0); // inicjujemy 0

...
    
    public LiveData<Integer> getScore() {
        return score;
    }

    public LiveData<Integer> getCurrentWordCount() {
        return currentWordCount;
    }

Zmodyfikujmy metody
- `nextWord` - zwróćmy uwagę na wywołanie metody `getValue` - tutaj musimy sprawdzić czy nie zwróci ona `null`

In [ ]:
    public boolean nextWord() {
            if (currentWordCount.getValue() != null &&
                    currentWordCount.getValue() < DataProvider.MAX_NO_OF_WORDS) {
                getNextWord();
                return true;
            }
            return false;
    }

- `reinitializeData`

In [ ]:
    public void reinitializeData() {
        score.setValue(0);
        currentWordCount.setValue(0);
        usedWordsList.clear();
        getNextWord();
    }

- w metodzie `getNextWord` musimy wykonać inkrementację, nie możemy zrobić tego wprost

In [ ]:
    if (currentWordCount.getValue() != null)
        currentWordCount.setValue(currentWordCount.getValue() + 1);

- podobnie w metodzie `increaseScore`

In [ ]:
    private void increaseScore() {
        if (score.getValue() != null)
            score.setValue(score.getValue() + DataProvider.SCORE_INCREASE);
    }

Przejdźmy do `ScrambleFragment`, w pierwszej kolejności pozbądźmy się metody `updateNextWordOnScreen` i wszystkich jej wywołań - nie będzie już ona potrzebna. Zmodyfikujmy metodę `onSubmitWord`

In [ ]:
    private void onSubmitWord() {
        String playerWord = "";
        if (binding.textInputEditText.getText() != null)
            playerWord = binding.textInputEditText.getText().toString();

        if (viewModel.isUserWordCorrect(playerWord)) {
            setErrorTextField(false);
            if (!viewModel.nextWord()) showFinalScoreDialog();
        } else setErrorTextField(true);
    }

Następnie podłączmy obserwator dla trzech `LiveData` w metodzie `onViewCreated`

In [ ]:
        viewModel.getCurrentScrambledWord()
            .observe(getViewLifecycleOwner(), newWord ->
                binding.textViewUnscrambledWord.setText(newWord));

        viewModel.getScore()
            .observe(getViewLifecycleOwner(), score ->
                binding.score.setText(String.valueOf(score)));

        viewModel.getCurrentWordCount()
            .observe(getViewLifecycleOwner(), wordCount ->
                binding.wordCount.setText(
                    getString(
                        R.string.word_count,
                        wordCount, 
                        DataProvider.MAX_NO_OF_WORDS)));

Metoda `observe` przyjnmuje dwa argumenty
- `LifeCycleOwner` - otrzymujemy przez wywołanie metody `getViewLifecycleOwner`
- lambda (obiekt anonimowy) - parametrem jest tutaj wartość "obserwowana" - zawsze aktualna

Pozostałe metody odpowiednio modysikujemy wykorzystując metody `getValue`

Możemy przetestować aplikację - zwróćmy uwagę że teraz wszystkie elementy layoutu są aktualizowane - ponieważ wykorzystujemy `ViewModel` i `LiveData` nie musimy implementować odpowiednich metod w klasie fragmentu.

<img src="https://media1.giphy.com/media/tcOAkShGdPpbMpwlMw/giphy.gif" width="200" />